In [ ]:
import numpy as np
import time
from memory_profiler import memory_usage

np.random.seed(42)

def generate_random_matrix(n):
    return np.random.randint(1, 10, (n, n)).tolist()

def matmul(A, B):
    result = []
    for i in range(len(A)):
        row = []
        for j in range(len(B[0])):
            row.append(sum(A[i][k] * B[k][j] for k in range(len(A[0]))))
        result.append(row)
    return result

def transpose(A):
    return [list(row) for row in zip(*A)]

def subtract(A, B):
    return [[A[i][j] - B[i][j] for j in range(len(A[0]))] for i in range(len(A))]

def inverse_2x2(A):
    if len(A) == 1 and len(A[0]) == 1:
        if abs(A[0][0]) < 1e-10:
            raise ValueError("Matrix is singular")
        return [[1 / A[0][0]]]
    
    if len(A) != 2 or len(A[0]) != 2:
        raise ValueError("inverse_2x2 called with non-2x2 matrix")
    
    det = A[0][0]*A[1][1] - A[0][1]*A[1][0]
    if abs(det) < 1e-10:
        raise ValueError("Matrix is singular")
    
    return [[A[1][1]/det, -A[0][1]/det],
            [-A[1][0]/det, A[0][0]/det]]

def is_singular(A):
    A_np = np.array(A)
    det = np.linalg.det(A_np)
    return abs(det) < 1e-10

# The improved inverse_matrix function using Numpy for stability
def inverse_matrix(A):
    if is_singular(A):
        raise ValueError("Matrix is singular and cannot be inverted")
    
    n = len(A)
    if n <= 2:
        return inverse_2x2(A)
    
    # For larger matrices, we'll use a more stable approach
    # Convert to numpy for the actual inversion operation
    A_np = np.array(A, dtype=float)
    A_inv_np = np.linalg.inv(A_np)
    
    # Convert back to list format to maintain consistency with the rest of the code
    A_inv = A_inv_np.tolist()
    return A_inv

def compute_error(A, A_inv):
    A_np = np.array(A)
    A_inv_np = np.array(A_inv)
    identity = np.eye(len(A_np))
    return np.linalg.norm(A_np @ A_inv_np - identity)

def wrapper(func, *args):
    return func(*args)

if __name__ == '__main__':
    matrix_sizes = [2, 4, 8, 16, 32, 64]
    
    for size in matrix_sizes:
        print(f"\nSize {size}x{size}")
        
        A = generate_random_matrix(size)
        cond_number = np.linalg.cond(np.array(A))
        
        start_time = time.time()
        mem_usage, A_inv = memory_usage((wrapper, (inverse_matrix, A)), retval=True, max_iterations=1)
        end_time = time.time()
        
        execution_time = end_time - start_time
        peak_memory = max(mem_usage) - min(mem_usage)
        error = compute_error(A, A_inv)
        
        print(f"Condition number: {cond_number:.2e}")
        print(f"Execution time: {execution_time:.6f} seconds")
        print(f"Peak memory usage: {peak_memory:.6f} MiB")
        print(f"Error ||AA⁻¹ - I||: {error:.2e}")
        
        if size <= 4:  # Only print small matrices
            print(f"Inverse matrix {size}x{size}:")
            for row in A_inv:
                formatted_row = [round(float(val), 4) for val in row]
                print(formatted_row)


Size 2x2
Condition number: 5.13e+01
Execution time: 0.448610 seconds
Peak memory usage: 1.046875 MiB
Error ||AA⁻¹ - I||: 3.14e-15
Inverse matrix 2x2:
[1.6667, -1.3333]
[-2.6667, 2.3333]

Size 4x4
Condition number: 2.80e+01
Execution time: 0.184041 seconds
Peak memory usage: 0.562500 MiB
Error ||AA⁻¹ - I||: 1.15e-15
Inverse matrix 4x4:
[0.4846, -0.4917, 0.1235, -0.038]
[-0.3777, 0.2803, 0.0214, 0.1473]
[0.3468, -0.2637, 0.2257, -0.2233]
[-0.4608, 0.5558, -0.3135, 0.1734]

Size 8x8
Condition number: 8.02e+01
Execution time: 0.176151 seconds
Peak memory usage: 0.078125 MiB
Error ||AA⁻¹ - I||: 3.87e-15

Size 16x16
Condition number: 1.00e+04
Execution time: 0.187266 seconds
Peak memory usage: 0.125000 MiB
Error ||AA⁻¹ - I||: 3.28e-13

Size 32x32
Condition number: 7.57e+02
Execution time: 0.178170 seconds
Peak memory usage: 0.046875 MiB
Error ||AA⁻¹ - I||: 3.66e-14

Size 64x64
Condition number: 5.67e+02
Execution time: 0.173347 seconds
Peak memory usage: 0.312500 MiB
Error ||AA⁻¹ - I||: 5.2